In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import pandas as pd
from skimage import io
from PIL import Image
from skimage import io


# from customDataset import CatsAndDogs

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [7]:

class CatsAndDogs(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index,0])
        # image = io.imread(img_path)
        image = Image.open(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index,1]))

        if self.transform:
            image = self.transform(image)
            return (image, y_label)

In [8]:
in_channel = 3
num_classes = 2
learning_rate = 0.01
batch_size = 32
num_epochs = 1

data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ColorJitter(),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(128),
    transforms.ToTensor()
])

train_set = CatsAndDogs(csv_file='cats_dogs.csv', root_dir = 'data/train1/train1', transform = data_transform)
# test_set = CatsAndDogs(csv_file='trainData.csv', root_dir = 'data/test/test', transform = transforms.ToTensor())
print(train_set)
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

# dataset = datasets.ImageFolder("./data/train1", transform=transforms)
# testset = datasets.ImageFolder("./data/test", transform=transforms)
# print(dataset)
# print(testset)

# train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)
# print(len(train_loader))



2
2
2


In [5]:
# Pretrained model
model = models.densenet121(pretrained=True)
model.to(device)

# Optimizer and Loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)


# Training
for epoch in range(num_epochs):
    losses = []
    print(f'Epoch {epoch+1}')
    for batch_idx, (data, target) in  enumerate(train_loader):
        data = data.to(device=device)
        target = target.to(device=device) # necessary for gpu
        
        print(f'Batch {batch_idx+1}')
        # forwardpropogation
        op = model(data)
        loss = criterion(op,target)

        losses.append(loss.item())

        # backpropogation
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        print(str(losses[-1]))
    print(f'Cost at epoch {epoch} is {sum(losses)/len(losses)}')

.7023749351501465
Batch 92
1.1445685625076294
Batch 93
0.6861129403114319
Batch 94
0.6391443610191345
Batch 95
0.6706876754760742
Batch 96
0.6636170744895935
Batch 97
0.7197605967521667
Batch 98
0.6542128324508667
Batch 99
0.6745536923408508
Batch 100
0.5912331342697144
Batch 101
1.3799697160720825
Batch 102
0.6071480512619019
Batch 103
0.8248482346534729
Batch 104
0.8459669947624207
Batch 105
0.967266321182251
Batch 106
0.8952287435531616
Batch 107
0.7432523965835571
Batch 108
1.3445173501968384
Batch 109
1.104207158088684
Batch 110
0.7733722925186157
Batch 111
0.8045117259025574
Batch 112
0.742232620716095
Batch 113
0.7338942885398865
Batch 114
0.950824499130249
Batch 115
0.7458351254463196
Batch 116
0.7623037099838257
Batch 117
0.9705656170845032
Batch 118
0.8922450542449951
Batch 119
0.6952439546585083
Batch 120
0.6039553284645081
Batch 121
0.6422441005706787
Batch 122
1.0854243040084839
Batch 123
1.0031747817993164
Batch 124
0.6812429428100586
Batch 125
0.6599283218383789
Batch 12

In [10]:
torch.save(model.state_dict(), 'C:\Projects\dogsvscats\weights.pth')

In [14]:
def calc_accuracy(loader,model):
    correct = 0
    samples = 0
    i = 0
    j=10
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x=x.to(device)
            y=y.to(device)

            op = model(x)
            _,predictions = op.max(1)
            correct += (predictions == y).sum()
            samples += predictions.size(0)
            i += 1
            if(i%10==0):
                print(f'calculating batch {j}')
                i=0
                j += 10
        print(f"Got {correct} / {samples} with accuracy {float(correct)/float(samples)*100:.2f}")
    model.train()

In [15]:
# Checking accuracies
# Training set
model_path = "C:\Projects\dogsvscats\weights.pth"
model = torch.load(model_path, map_location=torch.device(device) )
    for parameter in model.parameters():
        parameter.requires_grad = False

print("Accuracy of traning Set : ")
calc_accuracy(train_loader,model)
# Test set
print("Accuracy of test Set : ")
calc_accuracy(test_loader,model)
 

Accuracy of traning Set : 


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'